<a href="https://colab.research.google.com/github/Siliconvalley4uYouthProjects/SpringBoard-Swatcloud/blob/main/Data_Preprocessing_Industry_Labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing libraries

import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive
#drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#This code should be replaced with the correct path that contains the job data.
df = pd.read_csv('/content/drive/MyDrive/Swatcloud/Data/web_scraper/all_job_categories/Output Files/combined_all_companies_all_jobs.csv',header=None,names=['Company','Job Title','Job Link','Job Description'],skiprows=1)

# Text Cleaning tasks
# Removing new line characters
df['Job Description'] = df['Job Description'].astype(str)
df['Job Description'] = df['Job Description'].apply(lambda x: x.replace('\n', ' '))
# Removing empty leading and trailing spaces 
df['Job Description'] = df['Job Description'].apply(lambda x: x.strip())
# Removing special characters
df['Job Description'] = df['Job Description'].replace(r'[^\w\s]+', '', regex=True)
df['Job Description'] = df['Job Description'].replace(r'[^a-zA-Z0-9]', ' ', regex=True)
# Converting the text to lowercase
df['Job Description'] = df['Job Description'].str.lower()
# Splitting each word
df['Job Description'] = df['Job Description'].apply(lambda x: x.split(' '))

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stopwords = list(stopwords.words('english'))
stopwords[0:5]
lemmatizer = WordNetLemmatizer()

for i in range(len(df['Job Description'])):
    text = []
    for word in df['Job Description'][i]:
        if word not in stopwords:
            word = lemmatizer.lemmatize(word)
            text.append(word)
    df['Job Description'][i] = text
df['Job Description'] = df['Job Description'].apply(lambda x: ' '.join(x))
df['Job Description'] = df['Job Description'].replace(r' +', ' ', regex=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
agg = []
for title in df.iloc[:,1]:
    agg.append(title)

In [ ]:
#Counts of different words in the job titles
vectorizer = CountVectorizer(ngram_range = (1,4),stop_words = 'english')
cv = vectorizer.fit_transform(agg)
counts = pd.DataFrame(vectorizer.fit_transform(agg).sum(axis=0),
                      columns=vectorizer.get_feature_names())
counts = counts.T
common_words = counts[0].sort_values(ascending=False)[:50]
print(common_words)

manager              2676
engineer             1768
senior               1762
associate            1093
software              936
tax                   719
management            679
business              662
data                  630
cloud                 578
services              552
intern                547
software engineer     542
engineering           541
specialist            536
consultant            506
lead                  501
program               486
2023                  482
analyst               481
federal               442
sr                    441
product               437
development           405
sales                 398
developer             389
google                357
senior associate      343
ii                    342
technician            337
technical             317
google cloud          301
program manager       297
operations            297
director              287
senior manager        279
center                261
private               252
summer      

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
#Labeling the software engineering & tech related jobs

import nltk
swe_keywords = ['engineer','software','development','developer','engineering','system']
bus_keywords = ['operations','business','finance','audit','risk','tax','accounting']
data_keywords = ['data','analyst','machine','ai','ml','analytics']
marketing_keywords = ['media','ux','marketing','brand','social','event','advertise','campaign']
sales_keywords = ['sales','customer','sale','transaction']
research_keywords = ['research','scientist','researcher']
it_keywords = ['cyber','security','it','architect']

df['Industry'] = pd.Series()
for i in range(len(df)):
  k = 0
  for word in df['Job Title'][i].lower().split(' '):
    word = ''.join(x for x in word if x.isalnum())
    if word in swe_keywords:
      df['Industry'][i] ='Software Engineering'
      break
    elif word in data_keywords:
      df['Industry'][i] ='Data Analysis'
      break
    elif word in marketing_keywords:
      df['Industry'][i] = 'Marketing'
      break
    elif word in sales_keywords:
      df['Industry'][i] = 'Sales'
      break
    elif word in bus_keywords:
      df['Industry'][i] ='Operations & Finance'
      break
    elif word in research_keywords:
      df['Industry'][i] = 'Research Scientist'
      break
    elif word in it_keywords:
      df['Industry'][i] = 'Cybersecurity & IT'
      break
    else:
      k += 1
    if k == len(df['Job Title'][i].lower().split(' ')):
      #This means that there were no keywords found in the entire job title
      df['Industry'][i] ='Other'
      break  

#Second Step: Loop through the job descriptions and assign categories based on the most common keywords
index_values = []
for idx in df.loc[df['Industry']=='Other','Job Description'].index:
  index_values.append(idx)
for i in index_values:
  swe_counter = 0
  data_counter = 0
  marketing_counter = 0
  sales_counter = 0
  bus_counter = 0
  research_counter = 0
  it_counter = 0
  for word in df['Job Description'][i].lower().split(' '):
    word = ''.join(x for x in word if x.isalnum())
    if word in swe_keywords:
      swe_counter += 1
    if word in data_keywords:
      data_counter += 1
    if word in marketing_keywords:
      marketing_counter += 1
    if word in sales_keywords:
      sales_counter += 1
    if word in bus_keywords:
      bus_counter += 1
    if word in research_keywords:
      research_counter += 1
    if word in it_keywords:
      it_counter += 1
  counters = [swe_counter,data_counter,marketing_counter,sales_counter,bus_counter,research_counter,it_counter]
  if max(counters) == swe_counter:
    df['Industry'][i] ='Software Engineering'
  elif max(counters) == data_counter:
    df['Industry'][i] ='Data Analysis'
  elif max(counters) == marketing_counter:
    df['Industry'][i] = 'Marketing'
  elif max(counters) == sales_counter:
    df['Industry'][i] = 'Sales'
  elif max(counters) == bus_counter:
    df['Industry'][i] = 'Operations & Finance'
  elif max(counters) == research_counter:
    df['Industry'][i] = 'Research Scientist'
  elif max(counters) == it_counter:
    df['Industry'][i] = 'Cybersecurity & IT'

df['Industry'].value_counts()

<ipython-input-5-79b2c0929862>:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['Industry'] = pd.Series()
<ipython-input-5-79b2c0929862>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Industry'][i] ='Technology (Non-SWE)'
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Software Engineering    4458
Operations & Finance    2935
Data Analysis           1298
Sales                    996
Cybersecurity & IT       478
Marketing                338
Research Scientist       284
Name: Industry, dtype: int64

In [ ]:
df.to_csv('jobs_labeled.csv')

Note that the cells starting from below until the end of the notebook is all legacy code from production when the technology jobs and marketing/business jobs were being saved in separate csv files. If the job data is loaded as one merged file from the beginning, this code below is unnecessary.

In [ ]:
#Labeling the marketing & business related jobs - Note that this is legacy code from production when the technology jobs and marketing/business related jobs
#were saved in separate csv files. If the data is loaded as one merged file, this code below is unnecessary.
df2 = pd.read_csv('marketing_job_descriptions.csv',header=None,names=['Industry','Company','Job Title','Job Description'],skiprows=1)
cisco_m = pd.read_csv('cisco_jobs_marketing_cleaned.csv',header=None,names=['Job Title','Job Description','Company','Industry'],skiprows=1)
cisco_m.reset_index(inplace=True,drop=True)
cisco_m.drop(columns=['Industry'],inplace=True)
df2 = pd.concat([df2,cisco_m])
df2['Job Description'] = df2['Job Description'].astype(str)
df2.reset_index(inplace=True)

# Text Cleaning tasks
# Removing new line characters
df2['Job Description'] = df2['Job Description'].astype(str)
df2['Job Description'] = df2['Job Description'].apply(lambda x: x.replace('\n', ' '))
# Removing empty leading and trailing spaces 
df2['Job Description'] = df2['Job Description'].apply(lambda x: x.strip())
# Removing special characters
df2['Job Description'] = df2['Job Description'].replace(r'[^\w\s]+', '', regex=True)
df2['Job Description'] = df2['Job Description'].replace(r'[^a-zA-Z0-9]', ' ', regex=True)
# Converting the text to lowercase
df2['Job Description'] = df2['Job Description'].str.lower()
# Splitting each word
df2['Job Description'] = df2['Job Description'].apply(lambda x: x.split(' '))

for i in range(len(df2['Job Description'])):
    text = []
    for word in df2['Job Description'][i]:
        if word not in stopwords:
            word = lemmatizer.lemmatize(word)
            text.append(word)
    df2['Job Description'][i] = text
df2['Job Description'] = df2['Job Description'].apply(lambda x: ' '.join(x))
df2['Job Description'] = df2['Job Description'].replace(r' +', ' ', regex=True)

df2['Industry'] = pd.Series()

for i in range(len(df2)):
  k = 0
  for word in df2['Job Title'][i].lower().split(' '):
    word = ''.join(x for x in word if x.isalnum())
    if word in swe_keywords:
      df2['Industry'][i] ='Software Engineering'
      break
    elif word in data_keywords:
      df2['Industry'][i] ='Data Analysis'
      break
    elif word in marketing_keywords:
      df2['Industry'][i] = 'Marketing'
      break
    elif word in sales_keywords:
      df2['Industry'][i] = 'Sales'
      break
    elif word in bus_keywords:
      df2['Industry'][i] ='Operations & Finance'
      break
    elif word in research_keywords:
      df2['Industry'][i] = 'Research Scientist'
      break
    elif word in it_keywords:
      df2['Industry'][i] = 'Cybersecurity & IT'
      break
    else:
      k += 1
    if k == len(df2['Job Title'][i].lower().split(' ')):
      #This means that there were no keywords found in the entire job title
      df2['Industry'][i] ='Other Business Related'
      break  

index_values = []
for idx in df2.loc[df2['Industry']=='Other Business Related','Job Description'].index:
  index_values.append(idx)
for i in index_values:
  swe_counter = 0
  data_counter = 0
  marketing_counter = 0
  sales_counter = 0
  bus_counter = 0
  research_counter = 0
  it_counter = 0
  for word in df2['Job Description'][i].lower().split(' '):
    word = ''.join(x for x in word if x.isalnum())
    if word in swe_keywords:
      swe_counter += 1
    if word in data_keywords:
      data_counter += 1
    if word in marketing_keywords:
      marketing_counter += 1
    if word in sales_keywords:
      sales_counter += 1
    if word in bus_keywords:
      bus_counter += 1
    if word in research_keywords:
      research_counter += 1
    if word in it_keywords:
      it_counter += 1
  counters = [swe_counter,data_counter,marketing_counter,sales_counter,bus_counter,research_counter,it_counter]
  if max(counters) == swe_counter:
    df2['Industry'][i] ='Software Engineering'
  elif max(counters) == data_counter:
    df2['Industry'][i] ='Data Analysis'
  elif max(counters) == marketing_counter:
    df2['Industry'][i] = 'Marketing'
  elif max(counters) == sales_counter:
    df2['Industry'][i] = 'Sales'
  elif max(counters) == bus_counter:
    df2['Industry'][i] = 'Operations & Finance'
  elif max(counters) == research_counter:
    df2['Industry'][i] = 'Research Scientist'
  elif max(counters) == it_counter:
    df2['Industry'][i] = 'Cybersecurity & IT'

df2['Industry'].value_counts()

<ipython-input-9-f6f4198a72b6>:10: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df2['Industry'] = pd.Series()
<ipython-input-9-f6f4198a72b6>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Industry'][i] = 'Marketing'


Software Engineering    481
Operations & Finance    470
Sales                   380
Marketing               251
Data Analysis           142
Cybersecurity & IT       42
Research Scientist       14
Name: Industry, dtype: int64

In [ ]:
df2.to_csv('marketing_jobs_labeled.csv')

In [ ]:
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]
df

,Industry,index,Company,Job Title,Job Description
0,Software Engineering,0,Amazon,Senior Software Development Engineer,· 4+ years of professional software developmen...
1,Software Engineering,1,Amazon,Software Development Engineer - Payments,· programming experience with at least one mod...
2,Software Engineering,2,Amazon,Software Development Engineer - Fintech,bachelor’s degree in computer science or relat...
3,Software Engineering,3,Amazon,Software Development Engineer,1+ years of experience contributing to the sys...
4,Software Engineering,4,Amazon,"Embedded Software Development Engineer, Satell...",1+ years of experience contributing to the sys...
...,...,...,...,...,...
5289,Data Analysis,1146,Infosys,VPAS - Technology Analyst,process specialist - customer service\n\nin th...
5290,Sales,1147,Infosys,Process Specialist - Customer Service,poland
5291,Operations & Finance,1149,Infosys,Business Analyst - Life Insurance and Annuities,infosys mccamish systems––senior technical arc...
5292,Cybersecurity & IT,1150,Infosys,Progress Technology Architect,senior operations manager – infosys mccamish\n...


In [ ]:
#Combine both dataframes into one
df = pd.concat([df,df2])
df.reset_index(inplace=True)
df.drop(columns=['level_0','index'],inplace=True)

# Text Cleaning tasks
# Removing new line characters
df['Job Description'] = df['Job Description'].astype(str)
df['Job Description'] = df['Job Description'].apply(lambda x: x.replace('\n', ' '))
# Removing empty leading and trailing spaces 
df['Job Description'] = df['Job Description'].apply(lambda x: x.strip())
# Removing special characters
df['Job Description'] = df['Job Description'].replace(r'[^\w\s]+', '', regex=True)
df['Job Description'] = df['Job Description'].replace(r'[^a-zA-Z0-9]', ' ', regex=True)
# Converting the text to lowercase
df['Job Description'] = df['Job Description'].str.lower()
# Splitting each word
df['Job Description'] = df['Job Description'].apply(lambda x: x.split(' '))

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stopwords = list(stopwords.words('english'))
stopwords[0:5]
lemmatizer = WordNetLemmatizer()

for i in range(len(df['Job Description'])):
    text = []
    for word in df['Job Description'][i]:
        if word not in stopwords:
            word = lemmatizer.lemmatize(word)
            text.append(word)
    df['Job Description'][i] = text
df['Job Description'] = df['Job Description'].apply(lambda x: ' '.join(x))
df['Job Description'] = df['Job Description'].replace(r' +', ' ', regex=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
df['Industry'].value_counts()

Software Engineering    3703
Operations & Finance    1041
Data Analysis            861
Sales                    531
Cybersecurity & IT       414
Marketing                305
Research Scientist       219
Name: Industry, dtype: int64

In [ ]:
df.to_csv('swe_marketing_jobs.csv',index=False)